
# Use scikit-learn and AI lifecycle capabilities to predict Boston house prices with `ibm-watson-machine-learning`

This notebook contains steps and code to demonstrate support of AI Lifecycle features in Watson Machine Learning Service. It contains steps and code to work with [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository. It also introduces commands for getting model and training data, persisting model, deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.9.

<div class="alert alert-block alert-danger">
This notebook has been updated and tested in Cloud Pak for Data 4.0.7 classic notebook python 3.9 environment by WW Data and AI team on Apr 18, 2022.
</div>

## Learning goals

The learning goals of this notebook are:

-  Download an externally trained scikit-learn model with dataset.
-  Persist an external model in Watson Machine Learning repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.
-  Update previously persisted model.
-  Redeploy model in-place.
-  Scale deployment.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Download externally created scikit model and data](#download)
3.	[Persist externally created scikit model](#persistence)
4.	[Deploy and score in a Cloud](#scoring)
5.  [Persist new version of the model](#update_model)
6.  [Redeploy new version of the model](#redeploy)
7.  [Deployment scaling](#scale)
8.  [Clean up](#cleanup)
9.	[Summary and next steps](#summary)

## Updates:

1. Added sample code for running the same workflow in and outside of CP4D cluster.
2. Added the [version parameter](#version) to the scoring endpoint url, which is missing from ```client.deployments.get_scoring_href(created_deployment)```.
3. Added [sample python code](#restapi) for authenticating using [CPD Platform API](https://cloud.ibm.com/apidocs/cloud-pak-data#get-authorization-token) and sending POST requests to the endpoint url of the deployed model.

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `api_key`.

In [2]:
import os
username = 'admin'
api_key = '6OopuaKuxfFvb3PSWDtAVlLJwKI2zoJV4FDJWg7V'
#url = 'https://datafabric.ibmcloudpack.com:18407'

# if running inside CPD cluster, use the internal url
url = os.environ["RUNTIME_ENV_APSX_URL"]

In [3]:
wml_credentials = {
    "username": username,
    "apikey": api_key,
    "url": url,
    "instance_id": 'openshift',
    "version": '4.0'
}

Alternatively you can use `username` and `password` to authenticate WML services.

```
wml_credentials = {
    "username": ***,
    "password": ***,
    "url": ***,
    "instance_id": 'openshift',
    "version": '4.0'
}

```

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [10]:
!pip install -U ibm-watson-machine-learning

In [4]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Space%20management.ipynb).

You can use `list` method to print all existing spaces.

In [5]:
client.spaces.list(limit=10)

------------------------------------  ---------------------  ------------------------
ID                                    NAME                   CREATED
445d8b1d-df65-4662-a3a8-d383474f5c44  Deployment Space Demo  2022-04-13T20:13:34.311Z
------------------------------------  ---------------------  ------------------------


**Action**: Assign space ID below

In [6]:
space_id = '445d8b1d-df65-4662-a3a8-d383474f5c44'

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [7]:
client.set.default_space(space_id)

'SUCCESS'

<a id="download"></a>
## 2. Download externally created scikit model and data
In this section, you will download externally created scikit models and data used for training it.

In [9]:
! pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=b279e1c074c9136e520bbb59eaac9f57dbdca77b4d0ead7d045a8f2e1f59f7a0
  Stored in directory: /tmp/1000650000/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [10]:
import os
import wget

data_dir = 'BOSTON_HOUSE_PRICES_DATA'
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

model_path = os.path.join(data_dir, 'boston_regressor_0_23_1.tar.gz')
new_model_path = os.path.join(data_dir, 'boston_regressor_new_0_23_1.tar.gz')
if not os.path.isfile(model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/boston_house_price/model/boston_regressor_0_23_1.tar.gz", out=data_dir)  
if not os.path.isfile(new_model_path):
    wget.download("https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/boston_house_price/model/boston_regressor_new_0_23_1.tar.gz", out=data_dir)  

In [11]:
from sklearn import datasets
import pandas as pd

# see variable defination here: http://lib.stat.cmu.edu/datasets/boston
boston_data = datasets.load_boston()
boston_df = pd.DataFrame(boston_data.data)
boston_df.columns = boston_data.feature_names
boston_df['PRICE'] = boston_data.target

In [12]:
train_df = boston_df
test_df = boston_df.drop(['PRICE'], axis=1)

In [13]:
train_df.shape

(506, 14)

<a id="persistence"></a>
## 3. Persist externally created scikit model

In this section, you will learn how to store your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 3.1: Publish model

#### Publish model in Watson Machine Learning repository.

Define model name, autor name and email.

In [28]:
#client.software_specifications.list(100)

In [14]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.8")

In [15]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=model_path,
    meta_props=metadata,
    training_data=train_df)

Note: Warnings!! :  Software specification default_py3.8 specified for the wml_model is deprecated and will be removed in the future. We recommend you use runtime-22.1-py3.9 instead. For details see Supported Frameworks https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-supported-deployment-frameworks


### 3.2: Get model details

In [16]:
import json

published_model_uid = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "ab9e1b80-f2ce-592c-a7d2-4f2344f77194",
      "name": "default_py3.8"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "created_at": "2022-04-18T18:52:34.656Z",
    "id": "e591a0fb-7694-42df-85d8-adedcab62afe",
    "modified_at": "2022-04-18T18:52:36.326Z",
    "name": "scikit model",
    "owner": "1000330999",
    "resource_key": "0f29b98d-ce80-4fd2-b705-46acd4ba4584",
    "space_id": "445d8b1d-df65-4662-a3a8-d383474f5c44"
  },
  "system": {
    "warnings": [
      {
        "message": "Software specification default_py3.8 specified for the wml_model is deprecated and will be removed in the future. We recommend you use runtime-22.1-py3.9 instead. For details see Supported Frameworks https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-supported-deployment-frameworks"
      }
    ]
  }
}


### 3.3 Get all models

In [17]:
models_details = client.repository.list_models(limit=10)

------------------------------------  ------------  ------------------------  -----------------
ID                                    NAME          CREATED                   TYPE
e591a0fb-7694-42df-85d8-adedcab62afe  scikit model  2022-04-18T18:52:34.002Z  scikit-learn_0.23
------------------------------------  ------------  ------------------------  -----------------


<a id="scoring"></a>
## 4. Deploy and score deployed model

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client.

### 4.1: Create model deployment

#### Create online deployment for published model

In [18]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of the scikit model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'e591a0fb-7694-42df-85d8-adedcab62afe' started

#######################################################################################


initializing
Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='c6b8ec2c-06bd-4e28-a096-28644d9e90c1'
------------------------------------------------------------------------------------------------




**Note**: Here we use deployment url saved in published_model object. In next section, we show how to retrive deployment url from Watson Machine Learning instance.

In [19]:
deployment_uid = client.deployments.get_uid(created_deployment)

Now you can print an online scoring endpoint. 

In [20]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/c6b8ec2c-06bd-4e28-a096-28644d9e90c1/predictions


You can also list existing deployments.

In [21]:
client.deployments.list(limit=10)

------------------------------------  ------------------------------  -----  ------------------------
GUID                                  NAME                            STATE  CREATED
c6b8ec2c-06bd-4e28-a096-28644d9e90c1  Deployment of the scikit model  ready  2022-04-18T18:53:05.999Z
------------------------------------  ------------------------------  -----  ------------------------


### 4.2: Get deployment details

In [47]:
print(json.dumps(client.deployments.get_details(deployment_uid), indent=2))

Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
{
  "entity": {
    "asset": {
      "id": "04082ba3-80ae-4a9d-adc7-2be137b69508"
    },
    "custom": {},
    "deployed_asset_type": "model",
    "hardware_spec": {
      "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
      "name": "S",
      "num_nodes": 1
    },
    "name": "Deployment of the scikit model",
    "online": {},
    "space_id": "445d8b1d-df65-4662-a3a8-d383474f5c44",
    "status": {
      "online_url": {
        "url": "https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/05dea38a-45da-4748-af51-935529dc3ad9/predictions"
      },
      "serving_urls": [
        "https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/05dea38a-45da-4748-af51-935529dc3ad9/predictions"
  

### 4.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [22]:
score_0 = list(test_df.iloc[0])
score_1 = list(test_df.iloc[1])

In [24]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

Use ``client.deployments.score()`` method to run scoring.

In [25]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [26]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          30.0038433770168
        ],
        [
          25.025562379053415
        ]
      ]
    }
  ]
}


<a id="update_model"></a>
## 5. Persist new version of the model

In this section, you'll learn how to store new version of your model in Watson Machine Learning repository by using the Watson Machine Learning Client.

### 5.1: Publish new version of the model

Save the current model version. 

In [27]:
print(json.dumps(client.repository.create_model_revision(published_model_uid), indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "ab9e1b80-f2ce-592c-a7d2-4f2344f77194",
      "name": "default_py3.8"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "commit_info": {
      "committed_at": "2022-04-18T18:54:16.002Z"
    },
    "created_at": "2022-04-18T18:52:34.656Z",
    "id": "e591a0fb-7694-42df-85d8-adedcab62afe",
    "modified_at": "2022-04-18T18:52:36.326Z",
    "name": "scikit model",
    "owner": "1000330999",
    "resource_key": "0f29b98d-ce80-4fd2-b705-46acd4ba4584",
    "rev": "1",
    "space_id": "445d8b1d-df65-4662-a3a8-d383474f5c44"
  },
  "system": {
    "warnings": [
      {
        "message": "Software specification default_py3.8 specified for the wml_model is deprecated and will be removed in the future. We recommend you use runtime-22.1-py3.9 instead. For details see Supported Frameworks https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-supported-deployment-frameworks"
 

Define new model name and update model content. 

In [30]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'scikit model - updated'
}

published_model = client.repository.update_model(
    model_uid=published_model_uid,
    update_model=new_model_path,
    updated_meta_props=metadata
)

Save new model revision of the updated model. 

In [31]:
new_model_revision = client.repository.create_model_revision(published_model_uid)
print(json.dumps(new_model_revision, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "ab9e1b80-f2ce-592c-a7d2-4f2344f77194",
      "name": "default_py3.8"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "commit_info": {
      "committed_at": "2022-04-18T18:57:05.002Z"
    },
    "created_at": "2022-04-18T18:52:34.656Z",
    "id": "e591a0fb-7694-42df-85d8-adedcab62afe",
    "modified_at": "2022-04-18T18:56:21.830Z",
    "name": "scikit model - updated",
    "owner": "1000330999",
    "resource_key": "34ee0e26-4773-41af-904b-3ac42df17a38",
    "rev": "2",
    "space_id": "445d8b1d-df65-4662-a3a8-d383474f5c44"
  },
  "system": {
    "warnings": [
      {
        "message": "Software specification default_py3.8 specified for the wml_model is deprecated and will be removed in the future. We recommend you use runtime-22.1-py3.9 instead. For details see Supported Frameworks https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-supported-deployment-fra

**Note:** Model revisions can be identified by model `id` and `rev` number. 

Get model `rev` number from creation details:

In [32]:
rev_id = new_model_revision['metadata'].get('rev')

You can list existing revisions of the model. 

In [33]:
client.repository.list_models_revisions(published_model_uid)

--  ----------------------  ------------------------
ID  NAME                    CREATED
2   scikit model - updated  2022-04-18T18:52:34.656Z
1   scikit model            2022-04-18T18:52:34.656Z
--  ----------------------  ------------------------


### 5.2: Get model details

In [34]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

print(json.dumps(model_details, indent=2))

This method is deprecated, please use get_model_id()
{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "software_spec": {
      "id": "ab9e1b80-f2ce-592c-a7d2-4f2344f77194",
      "name": "default_py3.8"
    },
    "type": "scikit-learn_0.23"
  },
  "metadata": {
    "commit_info": {
      "committed_at": "2022-04-18T18:57:57.378Z"
    },
    "created_at": "2022-04-18T18:52:34.656Z",
    "id": "e591a0fb-7694-42df-85d8-adedcab62afe",
    "modified_at": "2022-04-18T18:57:05.400Z",
    "name": "scikit model - updated",
    "owner": "1000330999",
    "resource_key": "34ee0e26-4773-41af-904b-3ac42df17a38",
    "space_id": "445d8b1d-df65-4662-a3a8-d383474f5c44"
  },
  "system": {
    "warnings": [
      {
        "message": "Software specification default_py3.8 specified for the wml_model is deprecated and will be removed in the future. We recommend you use runtime-22.1-py3.9 instead. For details see Supported Frameworks https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=sp

<a id="redeploy"></a>
## 6. Redeploy new version of the model

In this section, you'll learn how to redeploy new version of the model by using the Watson Machine Learning Client.

### 6.1 Redeploy model

In [36]:
metadata = {
    client.deployments.ConfigurationMetaNames.ASSET: {
        "id": published_model_uid,
        "rev": rev_id
    }
}
    
updated_deployment = client.deployments.update(deployment_uid=deployment_uid, changes=metadata)

Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
Since ASSET is patched, deployment with new asset id/rev is being started. Monitor the status using deployments.get_details(deployment_uid) api


Wait for the deployment update: 

In [37]:
import time

status = None
while status not in ['ready', 'failed']:
    print('.', end=' ')
    time.sleep(2)
    deployment_details = client.deployments.get_details(deployment_uid)
    status = deployment_details['entity']['status'].get('state')
    
print("\nDeployment update finished with status: ", status)

. Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
. Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
. Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
. Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-ha

### 6.2 Get updated deployment details

In [38]:
print(json.dumps(client.deployments.get_details(deployment_uid), indent=2))

Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
{
  "entity": {
    "asset": {
      "id": "e591a0fb-7694-42df-85d8-adedcab62afe",
      "rev": "2"
    },
    "custom": {},
    "deployed_asset_type": "model",
    "hardware_spec": {
      "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
      "name": "S",
      "num_nodes": 1
    },
    "name": "Deployment of the scikit model",
    "online": {},
    "space_id": "445d8b1d-df65-4662-a3a8-d383474f5c44",
    "status": {
      "message": {
        "level": "warning",
        "text": "Successfully patched the asset."
      },
      "online_url": {
        "url": "https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/c6b8ec2c-06bd-4e28-a096-28644d9e90c1/predictions"
      },
      "serving_urls": [
        "https://in

<a id="scale"></a>
## 7. Deployment scaling

In this section, you'll learn how to scale your deployment by creating more copies of stored model with Watson Machine Learning Client.  
This feature is for providing High-Availability and to support higher throughput

### 7.1 Scale deployment

In this example, 2 deployment copies will be made.

In [39]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external scikit model - scaling",
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {
        "name": "S",
        "num_nodes": 2
    }
}

In [40]:
scaled_deployment = client.deployments.update(deployment_uid, metadata)

Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.


### 7.2 Get scaled deployment details

In [56]:
print(json.dumps(client.deployments.get_details(deployment_uid), indent=2))

Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
{
  "entity": {
    "asset": {
      "id": "e591a0fb-7694-42df-85d8-adedcab62afe",
      "rev": "2"
    },
    "custom": {},
    "deployed_asset_type": "model",
    "hardware_spec": {
      "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
      "name": "S",
      "num_nodes": 2
    },
    "name": "Deployment of external scikit model - scaling",
    "online": {},
    "space_id": "445d8b1d-df65-4662-a3a8-d383474f5c44",
    "status": {
      "message": {
        "level": "warning",
        "text": "scaling_status: completed;requested_copies: 2;deployed_copies: 2;more_info: Successfully patched the asset."
      },
      "online_url": {
        "url": "https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/c6b8ec2c-06

### 7.3 Score updated deployment

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [41]:
score_0 = list(test_df.iloc[0])
score_1 = list(test_df.iloc[1])

In [235]:
scoring_payload = {"input_data": [{"fields": test_df.columns.tolist(),"values": [score_0, score_1]}]}
print(scoring_payload)

{'input_data': [{'fields': ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'], 'values': [[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98], [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14]]}]}


or you can copy and paste this payload ```{"input_data": [{"values": [[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98], [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14]]}]}``` or ```{"input_data": [{"fields": ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"], "values": [[0.00632, 18.0, 2.31, 0.0, 0.538, 6.575, 65.2, 4.09, 1.0, 296.0, 15.3, 396.9, 4.98], [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.9, 9.14]]}]}```in the Test UI to test the deployed model.

#### Using REST API<a class="anchor" id="restapi"></a>

In [187]:
# if running inside CPD cluster
mltoken = os.environ['USER_ACCESS_TOKEN']

In [257]:
# if running outside of CPD cluster
#import requests, json
#url = 'https://datafabric.ibmcloudpack.com:18407/icp4d-api/v1/authorize'
#payload = '{"username":"admin","password":"CP4DDataFabric"}'
#eaders = {'content-type': 'application/json'}
##r = requests.post(url, data=payload, headers=headers, verify=False)

#mltoken = json.loads(r.content.decode("utf-8"))['token']

<a id="version"></a>

In [251]:
import re
cpd_url = "https://datafabric.ibmcloudpack.com:18407"
internal_endpoint_url = client.deployments.get_details(deployment_uid)['entity']['status']['online_url']['url'] + "?version=" + client.deployments.get_details(deployment_uid)['metadata']['created_at'][:10]
external_endpoint_url = cpd_url + internal_endpoint_url[list(re.finditer("/", internal_endpoint_url))[2].start():]

print("internal endpoint url: ", internal_endpoint_url)
print("external endpoint url: ", external_endpoint_url)

Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
Note: Software specification default_py3.8 is deprecated. Use runtime-22.1-py3.9 software specification instead when saving a model. For details, see https://www.ibm.com/docs/en/cloud-paks/cp-data/4.0?topic=specifications-software-hardware-deployments.
internal endpoint url:  https://internal-nginx-svc.cpd-instance.svc.cluster.local:12443/ml/v4/deployments/c6b8ec2c-06bd-4e28-a096-28644d9e90c1/predictions?version=2022-04-18
external endpoint url:  https://datafabric.ibmcloudpack.com:18407/ml/v4/deployments/c6b8ec2c-06bd-4e28-a096-28644d9e90c1/predictions?version=2022-04-18


In [193]:
import requests
header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# use external_endpoint url if using outside of CP4D cluster
response_scoring = requests.post(internal_endpoint_url, json=scoring_payload, headers=header)
print("Scoring response")
print(json.loads(response_scoring.text))

Scoring response
{'predictions': [{'fields': ['prediction'], 'values': [[25.340000000000003], [22.06799999999997]]}]}


Use client.deployments.score() method to run scoring.

In [194]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [195]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          25.340000000000003
        ],
        [
          22.06799999999997
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 8. Clean up 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.0/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 9. Summary and next steps

 You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. 
 
 Check out our <a href="https://dataplatform.cloud.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Daniel Ryszka**, Software Engineer  
**Jan Sołtysik**, Intern  
Updated by Catherine Cao

Copyright © 2020, 2021, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.